# Start


This script prepares vapor pressure deficit (VPD) data during the hottest and coldest time of a day  from SILO and matches it with VPD from field observations for subsequent analysis.


In [ ]:
working_dir = '../..'  # This repository's root directory
input_file_name = 'in-situ_topography_pcs.csv'
max_temp_output_file_name = 'silo_vpd_pcs_max_temp.csv'
min_temp_output_file_name = 'silo_vpd_pcs_min_temp.csv'

download_silo_data = True

In [ ]:
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

sys.path.append(working_dir)
from Utils.vpd import calculate_vpd

# Loading in-situ and remote data


In [ ]:
# Load in-situ_topography.csv as the main df

df = pd.read_csv(os.path.join(working_dir, "output", "csv", input_file_name))
df = df[~df['VPD'].isna()]
df['Datetime'] = pd.to_datetime(df['Datetime'])

first_date = min(df['Datetime']).strftime("%Y%m%d")
day_after_last_date = (max(df['Datetime']) + pd.Timedelta(days=1)).strftime("%Y%m%d")
print("First date: ", first_date, ", One day after the last date: ", day_after_last_date)

df.head()

In [ ]:
# Load silo data


def nearest_silo_cell(x, y):
    return (round(round(x / 0.05) * 0.05, 3), round(round(y / 0.05) * 0.05, 3))


# Make a list of locations of all sites and reduce it to silo grid cell locations
df['silo_X'] = df.apply(lambda row: nearest_silo_cell(row['X'], row['Y'])[0], axis=1)
df['silo_Y'] = df.apply(lambda row: nearest_silo_cell(row['X'], row['Y'])[1], axis=1)
silo_cell_locations_list = list(set((x, y) for x, y in df[['silo_X', 'silo_Y']].values))
print("silo_cell_locations_list: ", silo_cell_locations_list)

# Use Python loop and curl to download all silo data
silo_data_dir = os.path.join(working_dir, "Data", "silo")
if download_silo_data:
    os.makedirs(silo_data_dir, exist_ok=True)
    for x, y in silo_cell_locations_list:
        output_file_path = os.path.join(silo_data_dir, f"silo_data_{x}_{y}.csv")
        url = f"https://www.longpaddock.qld.gov.au/cgi-bin/silo/DataDrillDataset.php?lat={y}&lon={x}&format=csv&start={first_date}&finish={day_after_last_date}&username=noemail@net.com&dataset=Official&comment=xnhg"
        print(f"Downloading {url} to {output_file_path}")
        !curl -L "{url}" -o "{output_file_path}" -C -

# Combining in-situ and remote data into a single dataframe

In [ ]:
# Find in-situ data's hottest and coldest observation of each day

# Assign silo_observation_date to each observation
mask = (df['Datetime'].dt.hour > 9) | (
    (df['Datetime'].dt.hour == 9) & (df['Datetime'].dt.minute > 0)
)
df.loc[mask, 'silo_observation_date'] = df.loc[mask, 'Datetime'] + pd.Timedelta(days=1)
df.loc[~mask, 'silo_observation_date'] = df.loc[~mask, 'Datetime']
df['silo_observation_date'] = pd.to_datetime(df['silo_observation_date']).dt.date

# Remove rows of the first and last dates of each site because those days lack some hours
first_last_dates = df.groupby('SiteID')['silo_observation_date'].agg(['min', 'max']).reset_index()
df = df.merge(first_last_dates, on='SiteID', how='left')
df_date_filtered = df[
    (df['silo_observation_date'] != df['min']) & (df['silo_observation_date'] != df['max'])
]
df_date_filtered = df_date_filtered.drop(columns=['min', 'max'])

# Select only rows with max tempurature of the day
df_date_filtered['Day_max_temp'] = df_date_filtered.groupby(['SiteID', 'silo_observation_date'])[
    'Temperature'
].transform('max')
df_max_temp = df_date_filtered[df_date_filtered['Temperature'] == df_date_filtered['Day_max_temp']]
print("In-situ day maximum temperature observations:")
print(df_max_temp.head())
print('\n')

# Select only rows with min tempurature of the day
df_date_filtered['Day_min_temp'] = df_date_filtered.groupby(['SiteID', 'silo_observation_date'])[
    'Temperature'
].transform('min')
df_min_temp = df_date_filtered[df_date_filtered['Temperature'] == df_date_filtered['Day_min_temp']]
print("In-situ day minimum temperature observations:")
df_min_temp.head()

# -------------------------------------- Investigation code below --------------------------------------
# df_max_temp[df_max_temp['Datetime'].dt.hour < 9]

# import datetime
# df[(df['SiteID'] == 79) & (df['Date'] == datetime.date(2019, 1, 30))]
# df[(df['SiteID'] == 90) & (df['Date'] == datetime.date(2019, 2, 19))]  ## Hottest in the morning
# df[(df['SiteID'] == 90) & (df['Date'] == datetime.date(2019, 3, 6))]
# df[(df['SiteID'] == 142) & (df['Date'] == datetime.date(2019, 3, 27))]
# df[(df['SiteID'] == 78) & (df['Date'] == datetime.date(2019, 1, 19))]

# df_date_filtered.groupby(['SiteID', 'Date'])['Datetime'].apply(lambda x: x.nunique())

In [ ]:
# Use only one observation per sit per day

df_max_temp = df_max_temp.loc[
    df_max_temp.groupby(['SiteID', 'silo_observation_date'])['RH'].idxmin()
]
df_min_temp = df_min_temp.loc[
    df_min_temp.groupby(['SiteID', 'silo_observation_date'])['RH'].idxmax()
]
df_max_temp.head()

# Investigating the number of obs per site per day
# df_max_temp.groupby(['SiteID', 'silo_observation_date'])['Datetime'].apply(lambda x: x.nunique())
# Check the accuracy of the code
# import datetime
# df_max_temp[(df_max_temp['SiteID'] == 264) & (df_max_temp['silo_observation_date'] == datetime.date(2020, 3, 21))]

In [ ]:
# Fill in df with silo data at the hottest time of the day (max_temp, RH_tmax)


# For each row, open silo data csv file one-by-one to get data
def get_silo_value(row, column):
    file_path = os.path.join(silo_data_dir, f"silo_data_{row['silo_X']}_{row['silo_Y']}.csv")
    df_silo = pd.read_csv(file_path)

    df_silo['Datetime'] = pd.to_datetime(df_silo['YYYY-MM-DD'], format='%Y-%m-%d')
    target_date = row['silo_observation_date']
    matched = df_silo.loc[df_silo['Datetime'].dt.date == target_date, column]

    if matched.empty:
        print(
            f'Cannot find matched SILO data for in-situ observation SiteID {row['SiteID']} at {target_date}'
        )
        return None

    return matched.values[0]


df_max_temp['silo_Temperature'] = df_max_temp.progress_apply(
    lambda row: get_silo_value(row, 'max_temp'), axis=1
)
df_max_temp['silo_RH'] = df_max_temp.progress_apply(
    lambda row: get_silo_value(row, 'rh_tmax'), axis=1
)
df_max_temp.head()

In [ ]:
# Fill in df with silo data at the coldest time of the day (min_temp, RH_tmin)

df_min_temp['silo_Temperature'] = df_min_temp.progress_apply(
    lambda row: get_silo_value(row, 'min_temp'), axis=1
)
df_min_temp['silo_RH'] = df_min_temp.progress_apply(
    lambda row: get_silo_value(row, 'rh_tmin'), axis=1
)
df_min_temp.head()

In [ ]:
# Investigate null values. Site 251 is located in the ocean.
print("df_max_temp length: ", len(df_max_temp))
print("df_min_temp length: ", len(df_min_temp))
df_max_temp[df_max_temp.isna().any(axis=1)]
df_min_temp[df_min_temp.isna().any(axis=1)]

# Calculating remote VPD from remote temperature and remote relative humidity

In [ ]:
# Write a function for deriving VPD from temp and RH

df_max_temp['silo_VPD'] = df_max_temp.apply(
    lambda row: calculate_vpd(row['silo_Temperature'], row['silo_RH']), axis=1
)
df_min_temp['silo_VPD'] = df_min_temp.apply(
    lambda row: calculate_vpd(row['silo_Temperature'], row['silo_RH']), axis=1
)
df_max_temp.head()

# Save the resulting dataframes

In [ ]:
# Drop helper columns
df_max_temp.drop(columns=['Day_max_temp'], inplace=True)
df_min_temp.drop(columns=['Day_max_temp', 'Day_min_temp'], inplace=True)

# Remove observations without SILO data
df_max_temp = df_max_temp[~df_max_temp['silo_VPD'].isna()]
df_min_temp = df_min_temp[~df_min_temp['silo_VPD'].isna()]

In [ ]:
# save df
df_max_temp.to_csv(
    os.path.join(working_dir, "output", "csv", max_temp_output_file_name), index=False
)
df_min_temp.to_csv(
    os.path.join(working_dir, "output", "csv", min_temp_output_file_name), index=False
)